# Semantic Kernel
![Alt text](image-1.png)         ![Alt text](image.png)

![Alt text](image-2.png)

Install all python libraries

Prepare a semantic kernel instance first, loading also the AI service settings defined in the [Setup notebook](00-getting-started.ipynb):

In [ ]:
!python -m pip install semantic-kernel==0.3.10.dev0
!python -m pip install azure-devops
!python -m pip install semantic-kernel==0.3.10.dev0
!python -m pip install azure-search-documents==11.3.0b1 # you need to copy source as latest release also does not have all the models
#!python -m pip install azure-mgmt-search --pre --upgrade

Import Packages required

In [1]:
import os
from dotenv import dotenv_values
import semantic_kernel as sk
from semantic_kernel import ContextVariables, Kernel # Context to store variables and Kernel to interact with the kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion # AI services
from semantic_kernel.planning.sequential_planner import SequentialPlanner # Planner
from semantic_kernel.planning.basic_planner import BasicPlanner # Planner


kernel = sk.Kernel() # Create a kernel instance
kernel1 = sk.Kernel() #create another kernel instance for not having semanitc function in the same kernel 

useAzureOpenAI = True

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("chat_completion", AzureChatCompletion(deployment, endpoint, api_key))
    kernel1.add_chat_service("chat_completion", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))

In [4]:
deployment, api_key, endpoint

('ContosoDemos-OpenAI-gpt-35-turbo',
 'a33c3a92d6694dedad2bf1e2fb7a9c48',
 'https://ch1-open-ai.openai.azure.com/')

Import the **Semantic functions**:Azuredevops


In [19]:
# note: using skills from the samples folder
plugins_directory = "./plugins"

    # Import the semantic functions
DevFunctions=kernel1.import_semantic_skill_from_directory(plugins_directory, "AzureDevOps")



-Feature description - To genrate feature description based on predefined template


In [6]:
FDesFunction = DevFunctions["FeatureDescription"]    
resultFD = FDesFunction("Azure Resource Group Configuration Export and Infrastructure as Code (IAC) Generation")

print(resultFD)


Feature Title: Azure Resource Group Configuration Export and Infrastructure as Code (IAC) Generation

Description: This feature allows users to export Azure Resource Group configurations and generate Infrastructure as Code (IAC) templates for easy deployment and management of resources. It addresses the pain point of manual configuration and deployment of resources, which can be time-consuming and error-prone. This feature is significant to the product as it improves the efficiency and accuracy of resource management, reducing the risk of misconfiguration and downtime.

User Needs:
- Users need an easy way to export Azure Resource Group configurations.
- Users need a way to generate IAC templates for easy deployment and management of resources.
- Users need to reduce the risk of misconfiguration and downtime.

Functional Requirements:
- Ability to export Azure Resource Group configurations.
- Ability to generate IAC templates for easy deployment and management of resources.
- Support f

-Identify unser stories - Identify user stories from feature description

In [7]:
UDesFunction = DevFunctions["IdentifyUserStory"]    
resultUD = UDesFunction(str(resultFD))

print(resultUD)

"User Stories": [
{"UserStory" : "As a cloud administrator, I want to export Azure Resource Group configurations, so that I can easily manage and deploy resources.",
 "Description": "The user needs to be able to export the configurations of an Azure Resource Group in a simple and efficient way.",
 "Acceptance Criteria": Given an Azure Resource Group, when the user selects the export option, then a file containing the configurations is generated and downloaded."},
{"UserStory" : "As a cloud administrator, I want to generate ARM and Terraform IAC templates, so that I can easily deploy and manage resources.",
 "Description": "The user needs to be able to generate Infrastructure as Code templates in both ARM and Terraform formats.",
 "Acceptance Criteria": Given an Azure Resource Group, when the user selects the generate IAC templates option, then both ARM and Terraform templates are generated and downloaded."},
{"UserStory" : "As a cloud administrator, I want to customize IAC templates th

Importing and executing **Native plugin** - 
- Create- It creates feature in azure devops features with nested call to semantic function to genrate feature description

In [8]:
from plugins.AzureDevops.Feature_nested import feature
math_plugin = kernel.import_skill(feature(kernel1), skill_name="AzureDevOps")
variables = ContextVariables()
variables["title"] = "creating a nice pipelines"
variables["description"] = "test"
result = await kernel.run_async(
                math_plugin["create"], input_vars=variables
            )
print(result)

"Efficient Pipeline Creation" Feature created successfully


Using Basic planner to achive goal by sticking all the fuctions togather.

In [10]:
from plugins.AzureDevops.Feature_nested import feature
planner = BasicPlanner()
# Import the native functions
math_plugin = kernel.import_skill(feature(kernel1), skill_name="AzureDevOps")
ask = "create 5 Azure DevOps features based on transcript for IaC generation"
plan = await planner.create_plan_async(ask, kernel)
print(plan.generated_plan)

{
        "input": "IaC transcript",
        "subtasks": [
            {"function": "AzureDevOps.create", "args": {"description": "Feature 1", "title": "IaC Feature 1"}},
            {"function": "AzureDevOps.create", "args": {"description": "Feature 2", "title": "IaC Feature 2"}},
            {"function": "AzureDevOps.create", "args": {"description": "Feature 3", "title": "IaC Feature 3"}},
            {"function": "AzureDevOps.create", "args": {"description": "Feature 4", "title": "IaC Feature 4"}},
            {"function": "AzureDevOps.create", "args": {"description": "Feature 5", "title": "IaC Feature 5"}}
        ]
    }


In [29]:
import json

subtasks=json.loads(str(plan.generated_plan))
subtasks

{'input': 'IaC transcript',
 'subtasks': [{'function': 'AzureDevOps.create',
   'args': {'description': 'Feature 1', 'title': 'IaC Feature 1'}},
  {'function': 'AzureDevOps.create',
   'args': {'description': 'Feature 2', 'title': 'IaC Feature 2'}},
  {'function': 'AzureDevOps.create',
   'args': {'description': 'Feature 3', 'title': 'IaC Feature 3'}},
  {'function': 'AzureDevOps.create',
   'args': {'description': 'Feature 4', 'title': 'IaC Feature 4'}},
  {'function': 'AzureDevOps.create',
   'args': {'description': 'Feature 5', 'title': 'IaC Feature 5'}}]}

Excute plan based on steps from planner function

In [30]:
# Execute the plan
for index, subtask in enumerate(subtasks, start=1):
    result = await planner.execute_plan_async(plan, kernel)
    print("Plan results:")
    print(result)


Plan results:
"Automated Configuration Management"Feature created unsuccessfully
Plan results:
"Automated Configuration Management" Feature created successfully


**Sequential planner**

In [17]:
from plugins.AzureDevops.Feature_nested import feature
planner = SequentialPlanner(kernel)
# Import the native functions
math_plugin = kernel.import_skill(feature(kernel1), skill_name="AzureDevOps")
ask = "create two Azure DevOps features for one with title creating chai and one with creating work items with standard feature title and description"
plan = await planner.create_plan_async(goal=ask)
for step in plan._steps:
        print(step.description, ":", step._state.__dict__)


create a Azure DevOps feature with description : {'variables': {'input': ''}}
create a Azure DevOps feature with description : {'variables': {'input': ''}}


In [23]:

print("Plan results:")
result = await plan.invoke_async(ask)
for step in plan._steps:
        print(step.description, ":", step._state.__dict__)

Plan results:
create a Azure DevOps feature with description : {'variables': {'input': ''}}
create a Azure DevOps feature with description : {'variables': {'input': ''}}


**Action Planner**

In [25]:
from semantic_kernel.planning import ActionPlanner
planner = ActionPlanner(kernel)
ask = "create identify Azure DevOps features for one with title office work and one with self learning with standard feature title and description"
plan = await planner.create_plan_async(goal=ask)
for step in plan._steps:
        print(step.description, ":", step._state.__dict__)

 Execute sequential

In [28]:
result = await plan.invoke_async()


In [29]:
result

SKContext(memory=<semantic_kernel.memory.null_memory.NullMemory object at 0x0000029257662650>, variables=ContextVariables(variables={'input': '"Automated Email Reminders" Feature created successfully', 'description': 'Standard feature description.', 'title': 'office work'}), skill_collection=ReadOnlySkillCollection(data={}))

Importing packages required for preparing and using our our data

In [2]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureTextCompletion,
    AzureTextEmbedding,
)


from semantic_kernel.connectors.memory.azure_cognitive_search import (
    AzureCognitiveSearchMemoryStore,
)


Register chat completion end point and Embeded model for preparing memory and quering the data

In [3]:
kernel = sk.Kernel()

config = dotenv_values(".env")
COLLECTION_NAME = "carml-index-sample"

AZURE_COGNITIVE_SEARCH_ENDPOINT = config["AZURE_COGNITIVE_SEARCH_ENDPOINT"]
AZURE_COGNITIVE_SEARCH_ADMIN_KEY = config["AZURE_COGNITIVE_SEARCH_ADMIN_KEY"]
AZURE_OPENAI_API_KEY = config["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT = config["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_EM_DEPLOYMENT_NAME = config["AZURE_OPENAI_EM_DEPLOYMENT_NAME"]
AZURE_OPENAI_DEPLOYMENT_NAME = config["AZURE_OPENAI_DEPLOYMENT_NAME"]
vector_size = 1536

kernel.add_text_completion_service(
        "dv",
        AzureTextCompletion(
            deployment_name=AZURE_OPENAI_DEPLOYMENT_NAME,
            endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
        ),
    )
kernel.add_text_embedding_generation_service(
       "ada",
        AzureTextEmbedding(
            deployment_name=AZURE_OPENAI_EM_DEPLOYMENT_NAME,
            endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
        ),
    )

Initiating Endpoint for connecting with Azure cognitive search

In [4]:

kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
kernel.import_skill(sk.core_skills.TextMemorySkill())
connector = AzureCognitiveSearchMemoryStore(
        vector_size, AZURE_COGNITIVE_SEARCH_ENDPOINT, AZURE_COGNITIVE_SEARCH_ADMIN_KEY
    )

    # Register the memory store with the kernel
kernel.register_memory_store(memory_store=connector)


Populating information in cognitive serach using Embeding model

I have some CARML biceps which i am uploading in the cognitive search

In [5]:
folder_path = (r"C:\Users\vigarudi\Documents\code\Powershell\CARML\Main")  # Replace with the path to your folder

# List all files in the folder
file_list = os.listdir(folder_path)

# Iterate through each file and read its contents
for filename in file_list:
    file_path = os.path.join(folder_path, filename)
    
    # Check if it's a file (not a directory)
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            file_contents = file.read()
            # Now, you can work with the file_contents as a string
            await kernel.memory.save_information_async(
                COLLECTION_NAME, id=filename, text=file_contents
                )
            


AIException: (<ErrorCodes.ServiceError: 6>, 'OpenAI service failed to generate embeddings', InvalidRequestError(message="This model's maximum context length is 8191 tokens, however you requested 9631 tokens (9631 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", param=None, code=None, http_status=400, request_id=None))

searching the data in memory 

In [6]:
#serach in memory
questions = [
        "how to provision VM",
    ]

for question in questions:
    print(f"Question: {question}")
    result = await kernel.memory.search_async(COLLECTION_NAME, question)
    print(f"Answer: {result[0].text}\n")

Question: how to provision VM
Answer: param prefix string
param mainLocation string
param environment string
param identityResourceGroups array
param identityVirtualNetworks array
param identitySecurityGroups array
param identityRouteTables array
param enableADDSForest bool
param isDeployAvailbilityZone bool
param virtualMachines array
param identityKeyVaults array
param adminUser string
param adminPassword string
param _artifactsLocation string
param _artifactsLocationSasToken string

var createADDSForestModelUri = '${_artifactsLocation}CreateADDSForest.zip${_artifactsLocationSasToken}'
var addDCtoDomainModuleUri = '${_artifactsLocation}AddDCtoDomain.zip${_artifactsLocationSasToken}'

var ips = [for vm in virtualMachines: vm.identityVMPrivateIp]

targetScope = 'subscription'
//Identity resource Group deployment
module IdentityResourceGroupModule 'modules/resources/resource-group/main.bicep' = [for resourceGroupName in identityResourceGroups: {
  scope: subscription(resourceGroupName.s

In [ ]:

await connector.close_async()


embeding the search outcome with semantic function

In [35]:

sk_prompt = """
    Genrate bicep code to provision {{$user_input}} for example
    Example:
    - {{$fact1}} {{recall $fact1}} 
    - {{$fact1}}
    """.strip()



context = kernel.create_new_context()
context["fact1"] = "How to provision a VM?"
context[sk.core_skills.TextMemorySkill.COLLECTION_PARAM] = COLLECTION_NAME
context[sk.core_skills.TextMemorySkill.RELEVANCE_PARAM] = 0.8
context["user_input"] = "How to provision a VM?"
chat_func = kernel.create_semantic_function(sk_prompt, max_tokens=4000, temperature=0.8)
answer = await kernel.run_async(chat_func, input_vars=context.variables)
print(answer)

  
    - How to provision a VM?  
    - How to provision a VM?  
    - How to provision a VM?  
 
### ServiceNow
1. Create an application that will enable users to raise a ticket for a certain issue.  
2. Create an application that will enable users to book a meeting room. 
3. Create an application that will enable users to book a car park. 

### Azure Devops
1. Add a test task in your pipeline. 
2. Add a new environment in your pipeline. 
3. Add a new stage in your pipeline.  
4. Add a manual approval task in your pipeline. 
5. Add a new agent to the agent pool. 

### Azure
1. Create an azure function. 
2. Create a storage account. 
3. Create an availability set. 
4. Create a virtual machine. 
5. Create a virtual network. 

### Other
1. Provide an example of how you’ve worked collaboratively with developers in a previous role. 
2. Provide an example of how you’ve worked collaboratively with other departments in a previous role. 
3. Give an example of when you’ve identified an ineffici

Using Chat completion with our own data set

In [7]:
import requests
import json
url = "https://ch1-open-ai.openai.azure.com/openai/deployments/ContosoDemos-OpenAI-gpt-35-turbo/extensions/chat/completions?api-version=2023-06-01-preview"
headers =  {"Content-Type":"application/json","api-key": "a33c3a92d6694dedad2bf1e2fb7a9c48" }
data = '''{
    "dataSources": [
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": "https://contoso.search.windows.net",
                "key": "aAEeBL5XE83GVHfJqulOn4BBdRpZJ5J7fizBsAWL6jAzSeAf0yNt",
                "indexName": "carml-index-sample"
            }
        }
    ],
    "messages": [
        {
            "role": "user",
            "content": "Bicep file that contains code to deploy Azure resources, including virtual machines?"
        }
    ]
}'''
response = requests.post(url, data=data, headers=headers)
json_data = json.loads(response.text)
print(str(json_data["choices"][0]['messages'][0]['content']))

{"citations": [{"content": "param hubVnetPeerings array = []\n\ntargetScope = 'managementGroup'\n\n// vNet peerings between hub networks\n\n\n//vnet peering source to remote\nmodule hubVnetPeeringModule 'modules/network/virtual-network/virtual-network-peering/main.bicep' = [for hubVnetPeering in hubVnetPeerings: {\n  scope: resourceGroup(hubVnetPeering.sourceSubscriptionId, hubVnetPeering.sourceResourceGroup)\n  name: '${hubVnetPeering.sourceVnetName}-to-${hubVnetPeering.remoteVnetName}'\n  params: {\n    remoteVirtualNetworkId: resourceId(hubVnetPeering.remoteSubscriptionId, hubVnetPeering.remoteResourceGroup, 'Microsoft.Network/virtualNetworks', concat(hubVnetPeering.remoteVnetName))\n    localVnetName: hubVnetPeering.sourceVnetName\n    allowVirtualNetworkAccess: hubVnetPeering.allowVirtualNetworkAccess\n    allowForwardedTraffic: hubVnetPeering.allowForwardedTraffic\n    allowGatewayTransit: hubVnetPeering.allowGatewayTransit\n    useRemoteGateways: hubVnetPeering.useRemoteGateways